In [8]:
import networkx as nx
import pandas as pd
import multiprocessing as mp
import urllib  
import matplotlib.pyplot as plt                                        
LARGE_FILE = "https://raw.githubusercontent.com/Misterresearch/CUNY-Projects/master/tveanon_ip_guid.csv"
CHUNKSIZE = 100000 # processing 100,000 rows at time

def process_frame(df):
        # process data frame
        return len(df)

if __name__ == '__main__':
        reader = pd.read_csv(LARGE_FILE, chunksize=CHUNKSIZE)
        pool = mp.Pool(4) # use 4 processes

        funclist = []
        for df in reader:
                # process each data frame
                f = pool.apply_async(process_frame,[df])
                funclist.append(f)

        result = 0
        for f in funclist:
                result += f.get(timeout=10) # timeout in 10 seconds

#f = urllib.urlopen(LARGE FILE)
#myfile = f.read()
print f